In [178]:
from pathlib import Path

import networkx as nx
from pyvis.network import Network
import numpy as np
from scipy import stats

The response variable is total population size after SIMULATION_YEARS.

In [187]:
NUM_WORLD_LOCATIONS = 100
SIMULATION_YEARS = 10000
INITIAL_POPULATION_PROPORTION = 10000 
BIOMES = [
    "tropical and subtropical moist broadleaf forests",
    "tropical and subtropical dry broadleaf forests",
    "tropical and subtropical coniferous forests",
    "temperate broadleaf and mixed forests",
    "temperate coniferous forests",
    "boreal forests/taiga",
    "tropical and subtropical grasslands, savannas, and shrublands",
    "temperate grasslands, savannas, and shrublands",
    "flooded grasslands and savannas",
    "montane grasslands and shrublands",
    "tundra",
    "Mediterranean forests, woodlands, and scrub or sclerophyll forests",
    "deserts and xeric shrublands",
    "mangrove"
]

In [194]:
biomes = Network(
    directed=True,
    neighborhood_highlight=True, 
    select_menu=True, 
    filter_menu=True,
    cdn_resources="in_line"
)

pre_biomes = nx.Graph()

# https://stackoverflow.com/a/47555011/8423001
nodes_and_biomes_dict = {node_id: BIOMES[node_id] for node_id in range(len(BIOMES))}

pre_biomes.add_nodes_from(
    [(node, {"biome": attribute}) 
        for (node, attribute) 
        in nodes_and_biomes_dict.items()
    ] 
)

pre_biomes.add_edges_from(
    [
        (0, 1),
        (0, 2),
        (0, 3),
        (0, 4),
        (0, 6),
        (0, 9),
        (0, 12),
        (0, 13),
        (1, 2),
        (1, 6),
        (1, 8),
        (1, 9),
        (1, 12),
        (1, 13),
        (2, 3),
        (2, 4),
        (2, 6),
        (2, 8),
        (2, 9),
        (2, 12),
        (2, 13),
        (3, 4),
        (3, 5),
        (3, 6),
        (3, 7),
        (3, 8),
        (3, 9),
        (3, 11),
        (3, 12),
        (3, 13),
        (4, 5),
        (4, 6),
        (4, 7),
        (4, 8),
        (4, 9),
        (4, 10),
        (4, 11),
        (4, 12),
        (4, 13),
        (5, 7),
        (5, 8),
        (5, 9),
        (5, 10),
        (6, 7),
        (6, 8),
        (6, 9),
        (6, 12),
        (6, 13),
        (7, 8),
        (7, 9),
        (7, 11),
        (7, 12),
        (8, 9),
        (8, 11),
        (8, 12),
        (8, 13),
        (9, 11),
        (9, 12),
        (9, 13),
        (11, 12),
        (11, 13),
        (12, 13)
    ]
)

NodeDataView({0: {'biome': 'tropical and subtropical moist broadleaf forests'}, 1: {'biome': 'tropical and subtropical dry broadleaf forests'}, 2: {'biome': 'tropical and subtropical coniferous forests'}, 3: {'biome': 'temperate broadleaf and mixed forests'}, 4: {'biome': 'temperate coniferous forests'}, 5: {'biome': 'boreal forests/taiga'}, 6: {'biome': 'tropical and subtropical grasslands, savannas, and shrublands'}, 7: {'biome': 'temperate grasslands, savannas, and shrublands'}, 8: {'biome': 'flooded grasslands and savannas'}, 9: {'biome': 'montane grasslands and shrublands'}, 10: {'biome': 'tundra'}, 11: {'biome': 'Mediterranean forests, woodlands, and scrub or sclerophyll forests'}, 12: {'biome': 'deserts and xeric shrublands'}, 13: {'biome': 'mangrove'}})

In [180]:
world = Network(
    directed=True,
    neighborhood_highlight=True, 
    select_menu=True, 
    filter_menu=True,
    cdn_resources="in_line"
)

pre_world = nx.connected_watts_strogatz_graph(
    n=NUM_WORLD_LOCATIONS,
    k=5,
    p=0.5
)

In [181]:
for (v1, v2, weight) in pre_world.edges.data('weight'):
    # https://trenton3983.github.io/files/projects/2020-05-21_intro_to_network_analysis_in_python/2020-05-21_intro_to_network_analysis_in_python.html
    # https://stackoverflow.com/questions/40128692/networkx-how-to-add-weights-to-an-existing-g-edges

    # Here, the weights represent the ease of travelling between nodes.
    # A high weight indicates that travel is easy.
    pre_world[v1][v2]["weight"] = stats.expon.rvs(scale=1)

# Make the graph directed to indicate
# allowable population movements.
pre_world = pre_world.to_directed()

In [182]:
# Skip the last row because we only care about
# the upper triangle exclusive of th main diagonal
# of the adjacency matrix.
for v1 in range(NUM_WORLD_LOCATIONS - 1):
    for v2 in range(v1 + 1, NUM_WORLD_LOCATIONS):
        current_edge_data = pre_world.get_edge_data(v1, v2)
        if current_edge_data is None:
            # There is no need to update current_weight.
            continue

        # Extract weight attribute
        current_weight = current_edge_data["weight"]
        if current_weight < 1:
            # Make the ease of travel different
            # for one of the edges connecting the same
            # pair of nodes to simulate ocean currents.
            pre_world[v1][v2]["weight"] = stats.expon.rvs(scale=0.2)


In [183]:
betweenness_centralities = nx.betweenness_centrality(
    G=pre_world,
    weight="weight"
)

# Get a node with a maximal betweenness centrality.
# This node will hold our starting population.
# https://stackoverflow.com/a/280156/8423001
starting_node = max(betweenness_centralities, key=betweenness_centralities.get)

In [184]:
# Loop through nodes and set initial parameters.
for node in nx.nodes(G=pre_world):
    nx.set_node_attributes(
        G=pre_world, 
        values={node: {"carrying_capacity": 1000000}}
    )

In [185]:
world.from_nx(nx_graph=pre_world, show_edge_weights=True)
world.show("world.html")